# Assistants API Overview with Streaming (Python SDK)

The new [Assistants API](https://platform.openai.com/docs/assistants/overview) is a stateful evolution of our [Chat Completions API](https://platform.openai.com/docs/guides/text-generation/chat-completions-api) meant to simplify the creation of assistant-like experiences, and enable developer access to powerful tools like Code Interpreter, Retrieval and Function Calling. The API now also supports streaming, which means you can build real-time experiences for your UI.  

![Assistants API Diagram](../images/assistants_overview_diagram.png)

## Chat Completions API vs Assistants API

The primitives of the **Chat Completions API** are `Messages`, on which you perform a `Completion` with a `Model` (`gpt-3.5-turbo`, `gpt-4`, etc). It is lightweight and powerful, but inherently stateless, which means you have to manage conversation state, tool definitions, retrieval documents, and code execution manually.

In contrast, the primitives of the **Assistants API** are

- `Assistants`, which encapsulate a base model, instructions, tools, and (context) documents,
- `Threads`, which represent the state of a conversation, and
- `Runs`, which power the execution of an `Assistant` on a `Thread`, including textual responses and multi-step tool use.

We'll take a look at how these can be used to create powerful, stateful experiences.


## Setup

### Python SDK

> **Note**
> We've updated our [Python SDK](https://github.com/openai/openai-python) to add support for the Assistants API, so you'll need to update it to the latest version (`1.2.3` at time of writing).


In [ ]:
!pip install --upgrade openai

And make sure it's up to date by running:


In [2]:
!pip show openai | grep Version

Version: 1.2.3


### Pretty Printing Helper


In [4]:
import json

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

## Complete Example with Assistants API


### Assistants


The easiest way to get started with the Assistants API is through the [Assistants Playground](https://platform.openai.com/playground).


![Assistants Playground](../images/assistants_overview_assistants_playground.png)


Let's begin by creating an assistant! We'll create a Math Tutor just like in our [docs](https://platform.openai.com/docs/assistants/overview).


![Creating New Assistant](../images/assistants_overview_new_assistant.png)


You can view Assistants you've created in the [Assistants Dashboard](https://platform.openai.com/assistants).


![Assistants Dashboard](../images/assistants_overview_assistants_dashboard.png)


You can also create Assistants directly through the Assistants API, like so:


In [5]:
from openai import OpenAI
import os

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "<your OpenAI API key if not set as env var>"))


assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Answer questions briefly, in a sentence or less.",
    model="gpt-4-1106-preview",
)
show_json(assistant)

{'id': 'asst_2Hfaqc71l3fu7pvbd1PkPrpf',
 'created_at': 1711920054,
 'description': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': []}

Regardless of whether you create your Assistant through the Dashboard or with the API, you'll want to keep track of the Assistant ID. This is how you'll refer to your Assistant throughout Threads and Runs.


Next, we'll create a new Thread and add a Message to it. This will hold the state of our conversation, so we don't have re-send the entire message history each time.


### Threads


Create a new thread:


In [6]:
thread = client.beta.threads.create()
show_json(thread)

{'id': 'thread_LFIvmULMH7dUmorXIIcCmNEb',
 'created_at': 1711920081,
 'metadata': {},
 'object': 'thread'}

Then add the Message to the thread:


In [7]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?",
)
show_json(message)

{'id': 'msg_88FLsE82A89fyYRAYRuuuKWA',
 'assistant_id': None,
 'completed_at': None,
 'content': [{'text': {'annotations': [],
    'value': 'I need to solve the equation `3x + 11 = 14`. Can you help me?'},
   'type': 'text'}],
 'created_at': 1711920085,
 'file_ids': [],
 'incomplete_at': None,
 'incomplete_details': None,
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'status': None,
 'thread_id': 'thread_LFIvmULMH7dUmorXIIcCmNEb'}

> **Note**
> Even though you're no longer sending the entire history each time, you will still be charged for the tokens of the entire conversation history with each Run.

Notice how the Thread we created is **not** associated with the Assistant we created earlier! Threads exist independently from Assistants, which may be different from what you'd expect if you've used ChatGPT (where a thread is tied to a model/GPT).

### Streaming Runs

To get a completion from an Assistant for a given Thread, we must create a Run. Creating a Run will indicate to an Assistant that it should look at the messages in the Thread and take action: either by adding a single response, or using tools.

> **Note**
> Runs are a key difference between the Assistants API and Chat Completions API. While in Chat Completions the model will only ever respond with a single message, in the Assistants API a Run may result in an Assistant using one or multiple tools, and potentially adding multiple messages to the Thread.

In order to get the Assistant's response as a stream, we have to do a couple of things when setting up the run. First, we define an EventHandler class that specifies how we want to handle events in the response stream. Second, we use the 'create and stream' helper in the Python SDK to actually create the Run and stream the response. You only need to define EventHandler once, but every time you add a message to a thread and want to run it, you'd use the create_and_stream helper function again. 



In [9]:
from typing_extensions import override
from openai import AssistantEventHandler


class EventHandler(AssistantEventHandler):    
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)
      
  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)
      
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)
  
  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)



with client.beta.threads.runs.create_and_stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please be a friendly math tutor and explain all of your steps for solving a question.",
  event_handler=EventHandler(),
) as stream:
  stream.until_done()



assistant > Absolutely, I'd be happy to help you solve the equation `3x + 11 = 14`. The goal here is to isolate x (get x by itself on one side of the equation). Here are the steps to do that:

1. **Subtract 11 from both sides**: You want to move the constant term (11) from the left side to the right side to isolate the variable term (3x). To maintain the equality, whatever you do to one side, you must do to the other.

    3x + 11 - 11 = 14 - 11

    This simplifies to:

    3x = 3

2. **Divide both sides by 3**: Now that we have 3x on one side, we need to get x alone. Since x is being multiplied by 3, we can do the opposite operation, which is division by 3, to both sides of the equation.

    3x / 3 = 3 / 3

    This simplifies to:

    x = 1

You now have the solution to the equation, which is x = 1.

### Messages


Now that the Run has completed, we can list the Messages in the Thread to see what got added by the Assistant.


In [10]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
show_json(messages)

{'data': [{'id': 'msg_43Iu2C9eofqEesCULzGwjEyR',
   'assistant_id': 'asst_2Hfaqc71l3fu7pvbd1PkPrpf',
   'completed_at': None,
   'content': [{'text': {'annotations': [],
      'value': "Absolutely, I'd be happy to help you solve the equation `3x + 11 = 14`. The goal here is to isolate x (get x by itself on one side of the equation). Here are the steps to do that:\n\n1. **Subtract 11 from both sides**: You want to move the constant term (11) from the left side to the right side to isolate the variable term (3x). To maintain the equality, whatever you do to one side, you must do to the other.\n\n    3x + 11 - 11 = 14 - 11\n\n    This simplifies to:\n\n    3x = 3\n\n2. **Divide both sides by 3**: Now that we have 3x on one side, we need to get x alone. Since x is being multiplied by 3, we can do the opposite operation, which is division by 3, to both sides of the equation.\n\n    3x / 3 = 3 / 3\n\n    This simplifies to:\n\n    x = 1\n\nYou now have the solution to the equation, which is 

As you can see, Messages are ordered in reverse-chronological order – this was done so the most recent results are always on the first `page` (since results can be paginated). Do keep a look out for this, since this is the opposite order to messages in the Chat Completions API.


Let's ask our Assistant to explain the result a bit further!


In [11]:
# Create a message to append to our thread
message = client.beta.threads.messages.create(
    thread_id=thread.id, role="user", content="Could you explain this to me?"
)

# Execute our run again to process the new message
with client.beta.threads.runs.create_and_stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please be a friendly math tutor and explain all of your steps for solving a question.",
  event_handler=EventHandler(),
) as stream:
  stream.until_done()


# Retrieve all the messages added after our last user message
messages = client.beta.threads.messages.list(
    thread_id=thread.id, order="asc", after=message.id
)
show_json(messages)


assistant > Of course, I'll explain each step in more detail.

You're given the equation:

`3x + 11 = 14`

The goal is to solve for x, which means we want to find the value of x that makes this statement true. We need to isolate x on one side of the equation, which means we want to end up with an equation that looks like `x = ` some number.

Here's how we do it, step by step:

### Step 1: Subtract 11 from both sides.
We start by looking at the `+11` on the left side of the equation. We want to get rid of it so that we only have the term with x (the 3x) on that side. To do this, we do the opposite operation, which is subtraction. We subtract 11 from both sides of the equation:

`3x + 11 - 11 = 14 - 11`

On the left side, by subtracting 11 from 11, we get 0, so they cancel each other out. This leaves us with just `3x` on the left side.

On the right side, subtracting 11 from 14 gives us 3.

After performing the subtraction, the equation looks like this:

`3x = 3`

### Step 2: Divide bot

{'data': [{'id': 'msg_gFvDVfgTgGvQeGcipEGr4AkE',
   'assistant_id': 'asst_2Hfaqc71l3fu7pvbd1PkPrpf',
   'completed_at': None,
   'content': [{'text': {'annotations': [],
      'value': "Of course, I'll explain each step in more detail.\n\nYou're given the equation:\n\n`3x + 11 = 14`\n\nThe goal is to solve for x, which means we want to find the value of x that makes this statement true. We need to isolate x on one side of the equation, which means we want to end up with an equation that looks like `x = ` some number.\n\nHere's how we do it, step by step:\n\n### Step 1: Subtract 11 from both sides.\nWe start by looking at the `+11` on the left side of the equation. We want to get rid of it so that we only have the term with x (the 3x) on that side. To do this, we do the opposite operation, which is subtraction. We subtract 11 from both sides of the equation:\n\n`3x + 11 - 11 = 14 - 11`\n\nOn the left side, by subtracting 11 from 11, we get 0, so they cancel each other out. This leaves u

This may feel like a lot of steps to get a response back, especially for this simple example. However, you'll soon see how we can add very powerful functionality to our Assistant without changing much code at all!


### Example


Let's take a look at how we could potentially put all of this together. Below is all the code you need to use an Assistant you've created.

Since we've already created our Math Assistant, I've saved its ID in `MATH_ASSISTANT_ID`. I then defined two functions:

- `submit_message`: create a Message on a Thread, then start and return a new Run
- `get_response`: returns the list of Messages in a Thread


In [53]:
from openai import OpenAI
from openai import AssistantEventHandler
from typing_extensions import override
import os 

MATH_ASSISTANT_ID = assistant.id  # or a hard-coded ID like "asst-..."

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "<your OpenAI API key if not set as env var>"))

class EventHandler(AssistantEventHandler):    
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)
      
  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)
      
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)
  
  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)

def submit_message(assistant_id, thread, user_message):
    # Submit the initial message to the thread
    client.beta.threads.messages.create(
        thread_id=thread.id, role="user", content=user_message
    )

    # Create and stream the run with the given instructions
    with client.beta.threads.runs.create_and_stream(
        thread_id=thread.id,
        assistant_id=assistant_id,
        instructions="Please be a friendly math tutor.",
        event_handler=EventHandler(),
    ) as stream:
        stream.until_done()


def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id, order="asc")

I've also defined a `create_thread_and_run` function that I can re-use (which is actually almost identical to the [`client.beta.threads.create_and_run`](https://platform.openai.com/docs/api-reference/runs/createThreadAndRun) compound function in our API ;) ). Finally, we can submit our mock user requests each to a new Thread.

Notice how all of these API calls are asynchronous operations; this means we actually get async behavior in our code without the use of async libraries! (e.g. `asyncio`)


In [54]:
def create_thread_and_run(user_input):
    thread = client.beta.threads.create()
    run = submit_message(MATH_ASSISTANT_ID, thread, user_input)
    return thread, run


# Emulating concurrent user requests
thread1, run1 = create_thread_and_run(
    "I need to solve the equation `3x + 11 = 14`. Can you help me?"
)
thread2, run2 = create_thread_and_run("Can you generate another math problem like  `3x + 11 = 14` for me? Make it difficult.")

# Now all Runs are executing...


assistant > code_interpreter

from sympy import symbols, Eq, solve

# Define the variable and equation
x = symbols('x')
equation = Eq(3*x + 11, 14)

# Solving the equation
solution = solve(equation, x)
solution

output >

[1]

assistant > Certainly! To solve the equation \(3x + 11 = 14\), you'll need to isolate the variable \(x\) on one side of the equation.

Here's how we can do it step by step:
1. Subtract 11 from both sides to start.
\[ 3x + 11 - 11 = 14 - 11 \]
\[ 3x = 3 \]

2. Now, divide both sides by 3 to solve for \(x\).
\[ \frac{3x}{3} = \frac{3}{3} \]
\[ x = 1 \]

So the solution to the equation is \(x = 1\).
assistant > Sure, let's create a more challenging problem that involves multiple steps and possibly fractions or decimals. Here is an example:

Solve for \( x \) in the equation:

\[ 4x - \frac{5}{2} = \frac{3x}{4} + 9.5 \]

This problem involves fractions and requires multiple steps to solve, including finding a common denominator and moving terms with \( x \) to one s

Et voilà!

You may have noticed that this code is not actually specific to our math Assistant at all... this code will work for any new Assistant you create simply by changing the Assistant ID! That is the power of the Assistants API.


## Tools

A key feature of the Assistants API is the ability to equip our Assistants with Tools, like Code Interpreter, Retrieval, and custom Functions. Let's take a look at each.

### Code Interpreter

Let's equip our Math Tutor with the [Code Interpreter](https://platform.openai.com/docs/assistants/tools/code-interpreter) tool, which we can do from the Dashboard...


![Enabling code interpreter](../images/assistants_overview_enable_code_interpreter.png)


...or the API, using the Assistant ID.


In [36]:
assistant = client.beta.assistants.update(
    MATH_ASSISTANT_ID,
    tools=[{"type": "code_interpreter"}],
)
show_json(assistant)

{'id': 'asst_2Hfaqc71l3fu7pvbd1PkPrpf',
 'created_at': 1711920054,
 'description': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': [{'type': 'code_interpreter'}]}

Now, let's ask the Assistant to use its new tool.


In [40]:
thread, run = create_thread_and_run(
    "Generate the first 20 fibbonaci numbers with code."
)



assistant > code_interpreter

# Function to generate the first n Fibonacci numbers
def generate_fibonacci(n):
    fib_sequence = [0, 1]
    while len(fib_sequence) < n:
        fib_sequence.append(fib_sequence[-1] + fib_sequence[-2])
    return fib_sequence[:n]

# Generate the first 20 Fibonacci numbers
first_20_fib_numbers = generate_fibonacci(20)
first_20_fib_numbers

output >

[0,
 1,
 1,
 2,
 3,
 5,
 8,
 13,
 21,
 34,
 55,
 89,
 144,
 233,
 377,
 610,
 987,
 1597,
 2584,
 4181]

assistant > The first 20 numbers in the Fibonacci sequence are:

0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181.

And that's it! The Assistant used Code Interpreter in the background, and gave us a final response.

For some use cases this may be enough – however, if we want more details on what precisely an Assistant is doing we can take a look at a Run's Steps.

### Steps


A Run is composed of one or more Steps. Like a Run, each Step has a `status` that you can query. This is useful for surfacing the progress of a Step to a user (e.g. a spinner while the Assistant is writing code or performing retrieval).


In [42]:
run_steps = client.beta.threads.runs.steps.list(thread_id=thread.id, 
run_id=run.id, order="asc")

AttributeError: 'NoneType' object has no attribute 'id'

Let's take a look at each Step's `step_details`.


In [18]:
for step in run_steps.data:
    step_details = step.step_details
    print(json.dumps(show_json(step_details), indent=4))

{'tool_calls': [{'id': 'call_WMNqd63PtX8vZzTwaA6eWpBg',
   'code_interpreter': {'input': '# Python function to generate the first 20 Fibonacci numbers\ndef fibonacci(n):\n    fib_sequence = [0, 1]\n    while len(fib_sequence) < n:\n        fib_sequence.append(fib_sequence[-1] + fib_sequence[-2])\n    return fib_sequence\n\n# Generate the first 20 Fibonacci numbers\nfirst_20_fibonacci = fibonacci(20)\nfirst_20_fibonacci',
    'outputs': [{'logs': '[0,\n 1,\n 1,\n 2,\n 3,\n 5,\n 8,\n 13,\n 21,\n 34,\n 55,\n 89,\n 144,\n 233,\n 377,\n 610,\n 987,\n 1597,\n 2584,\n 4181]',
      'type': 'logs'}]},
   'type': 'code_interpreter'}],
 'type': 'tool_calls'}

null


{'message_creation': {'message_id': 'msg_z593lE5bvcD6BngeDFHDxzwm'},
 'type': 'message_creation'}

null


We can see the `step_details` for two Steps:

1. `tool_calls` (plural, since it could be more than one in a single Step)
2. `message_creation`

The first Step is a `tool_calls`, specifically using the `code_interpreter` which contains:

- `input`, which was the Python code generated before the tool was called, and
- `output`, which was the result of running the Code Interpreter.

The second Step is a `message_creation`, which contains the `message` that was added to the Thread to communicate the results to the user.


### Retrieval

Another powerful tool in the Assistants API is [Retrieval](https://platform.openai.com/docs/assistants/tools/knowledge-retrieval): the ability to upload files that the Assistant will use as a knowledge base when answering questions. This can also be enabled from the Dashboard or the API, where we can upload files we want to be used.


![Enabling retrieval](../images/assistants_overview_enable_retrieval.png)


In [43]:
# Upload the file
file = client.files.create(
    file=open(
        "data/language_models_are_unsupervised_multitask_learners.pdf",
        "rb",
    ),
    purpose="assistants",
)
# Update Assistant
assistant = client.beta.assistants.update(
    MATH_ASSISTANT_ID,
    tools=[{"type": "code_interpreter"}, {"type": "retrieval"}],
    file_ids=[file.id],
)
show_json(assistant)

{'id': 'asst_2Hfaqc71l3fu7pvbd1PkPrpf',
 'created_at': 1711920054,
 'description': None,
 'file_ids': ['file-wGSa82c3EhDR2onbkT8GxVSR'],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': [{'type': 'code_interpreter'}, {'type': 'retrieval'}]}

In [44]:
thread, run = create_thread_and_run(
    "What are some cool math concepts behind this ML paper pdf? Explain in two sentences."
)



assistant > retrieval


assistant > The ML paper discusses the use of a Transformer-based architecture leveraging a predictive model capable of generalizing from unordered tokens to structured patterns in data . It also mentions the Children's Book Test to evaluate the performance of language models on different categories of words, which provides insight into the mathematical modeling of language understanding and prediction of contextually relevant tokens .

> **Note**
> There are more intricacies in Retrieval, like [Annotations](https://platform.openai.com/docs/assistants/how-it-works/managing-threads-and-messages), which may be covered in another cookbook.


### Functions

As a final powerful tool for your Assistant, you can specify custom [Functions](https://platform.openai.com/docs/assistants/tools/function-calling) (much like the [Function Calling](https://platform.openai.com/docs/guides/function-calling) in the Chat Completions API). During a Run, the Assistant can then indicate it wants to call one or more functions you specified. You are then responsible for calling the Function, and providing the output back to the Assistant.

Let's take a look at an example by defining a `display_quiz()` Function for our Math Tutor.

This function will take a `title` and an array of `question`s, display the quiz, and get input from the user for each:

- `title`
- `questions`
  - `question_text`
  - `question_type`: [`MULTIPLE_CHOICE`, `FREE_RESPONSE`]
  - `choices`: ["choice 1", "choice 2", ...]

Unfortunately I don't know how to get user input within a Python Notebook, so I'll be mocking out responses with `get_mock_response...`. This is where you'd get the user's actual input.


In [45]:
def get_mock_response_from_user_multiple_choice():
    return "a"


def get_mock_response_from_user_free_response():
    return "I don't know."


def display_quiz(title, questions):
    print("Quiz:", title)
    print()
    responses = []

    for q in questions:
        print(q["question_text"])
        response = ""

        # If multiple choice, print options
        if q["question_type"] == "MULTIPLE_CHOICE":
            for i, choice in enumerate(q["choices"]):
                print(f"{i}. {choice}")
            response = get_mock_response_from_user_multiple_choice()

        # Otherwise, just get response
        elif q["question_type"] == "FREE_RESPONSE":
            response = get_mock_response_from_user_free_response()

        responses.append(response)
        print()

    return responses

Here's what a sample quiz would look like:


In [46]:
responses = display_quiz(
    "Sample Quiz",
    [
        {"question_text": "What is your name?", "question_type": "FREE_RESPONSE"},
        {
            "question_text": "What is your favorite color?",
            "question_type": "MULTIPLE_CHOICE",
            "choices": ["Red", "Blue", "Green", "Yellow"],
        },
    ],
)
print("Responses:", responses)

Quiz: Sample Quiz

What is your name?

What is your favorite color?
0. Red
1. Blue
2. Green
3. Yellow

Responses: ["I don't know.", 'a']


Now, let's define the interface of this function in JSON format, so our Assistant can call it:


In [47]:
function_json = {
    "name": "display_quiz",
    "description": "Displays a quiz to the student, and returns the student's response. A single quiz can have multiple questions.",
    "parameters": {
        "type": "object",
        "properties": {
            "title": {"type": "string"},
            "questions": {
                "type": "array",
                "description": "An array of questions, each with a title and potentially options (if multiple choice).",
                "items": {
                    "type": "object",
                    "properties": {
                        "question_text": {"type": "string"},
                        "question_type": {
                            "type": "string",
                            "enum": ["MULTIPLE_CHOICE", "FREE_RESPONSE"],
                        },
                        "choices": {"type": "array", "items": {"type": "string"}},
                    },
                    "required": ["question_text"],
                },
            },
        },
        "required": ["title", "questions"],
    },
}

Once again, let's update our Assistant either through the Dashboard or the API.


![Enabling custom function](../images/assistants_overview_enable_function.png)

> **Note**
> Pasting the function JSON into the Dashboard was a bit finicky due to indentation, etc. I just asked ChatGPT to format my function the same as one of the examples on the Dashboard :).


In [48]:
assistant = client.beta.assistants.update(
    MATH_ASSISTANT_ID,
    tools=[
        {"type": "code_interpreter"},
        {"type": "retrieval"},
        {"type": "function", "function": function_json},
    ],
)
show_json(assistant)

{'id': 'asst_2Hfaqc71l3fu7pvbd1PkPrpf',
 'created_at': 1711920054,
 'description': None,
 'file_ids': ['file-wGSa82c3EhDR2onbkT8GxVSR'],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': [{'type': 'code_interpreter'},
  {'type': 'retrieval'},
  {'function': {'name': 'display_quiz',
    'description': "Displays a quiz to the student, and returns the student's response. A single quiz can have multiple questions.",
    'parameters': {'type': 'object',
     'properties': {'title': {'type': 'string'},
      'questions': {'type': 'array',
       'description': 'An array of questions, each with a title and potentially options (if multiple choice).',
       'items': {'type': 'object',
        'properties': {'question_text': {'type': 'string'},
         'question_type': {'type': 'string',
          'enum': ['MULTIPLE_CHOICE', 'FREE_RESPONSE'

And now, we ask for a quiz.


In [49]:
thread, run = create_thread_and_run(
    "Make a quiz with 2 questions: One open ended, one multiple choice. Then, give me feedback for the responses."
)



assistant > retrieval


assistant > From the content of the provided document 'Language Models are Unsupervised Multitask Learners', we can formulate a quiz with one open-ended question and one multiple-choice question. 

Here are the drafted quiz questions:

**Open-Ended Question:**
What is the primary goal of advancing toward more general systems in the field of machine learning, according to the article?

**Multiple-Choice Question:**
Which one of the following tasks does the language model GPT-2 NOT achieve state of the art results in, according to a zero-shot setting in the document?
- A) Reading Comprehension
- B) Translation
- C) Summarization
- D) Language Modeling

Let's use the quiz tool to present these questions to the student.

Now, however, when we check the Run's `status` we see `requires_action`! Let's take a closer.


In [50]:
show_json(run)

AttributeError: 'NoneType' object has no attribute 'model_dump_json'

The `required_action` field indicates a Tool is waiting for us to run it and submit its output back to the Assistant. Specifically, the `display_quiz` function! Let's start by parsing the `name` and `arguments`.

> **Note**
> While in this case we know there is only one Tool call, in practice the Assistant may choose to call multiple tools.


In [51]:
# Extract single tool call
tool_call = run.required_action.submit_tool_outputs.tool_calls[0]
name = tool_call.function.name
arguments = json.loads(tool_call.function.arguments)

print("Function Name:", name)
print("Function Arguments:")
arguments

AttributeError: 'NoneType' object has no attribute 'required_action'

Now let's actually call our `display_quiz` function with the arguments provided by the Assistant:


In [52]:
responses = display_quiz(arguments["title"], arguments["questions"])
print("Responses:", responses)

NameError: name 'arguments' is not defined

Great! (Remember these responses are the one's we mocked earlier. In reality, we'd be getting input from the back from this function call.)

Now that we have our responses, let's submit them back to the Assistant. We'll need the `tool_call` ID, found in the `tool_call` we parsed out earlier. We'll also need to encode our `list`of responses into a `str`.


In [29]:
run = client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=[
        {
            "tool_call_id": tool_call.id,
            "output": json.dumps(responses),
        }
    ],
)
show_json(run)

{'id': 'run_98PGE3qGtHoaWaCLoytyRUBf',
 'assistant_id': 'asst_9HAjl9y41ufsViNcThW1EXUS',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1699828370,
 'expires_at': 1699828970,
 'failed_at': None,
 'file_ids': ['file-MdXcQI8OdPp76wukWI4dpLwW'],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'object': 'thread.run',
 'required_action': None,
 'started_at': 1699828370,
 'status': 'queued',
 'thread_id': 'thread_bICTESFvWoRdj0O0SzsosLCS',
 'tools': [{'type': 'code_interpreter'},
  {'type': 'retrieval'},
  {'function': {'name': 'display_quiz',
    'parameters': {'type': 'object',
     'properties': {'title': {'type': 'string'},
      'questions': {'type': 'array',
       'description': 'An array of questions, each with a title and potentially options (if multiple choice).',
       'items': {'type': 'object',
        'properties': {'question_text': {'type': 

We can now wait for the Run to complete once again, and check our Thread!


In [30]:
run = wait_on_run(run, thread)
pretty_print(get_response(thread))

# Messages
user: Make a quiz with 2 questions: One open ended, one multiple choice. Then, give me feedback for the responses.
assistant: Thank you for attempting the quiz.

For the first question, it's important to know that the square root of a negative number is not a real number because real numbers consist of all the numbers on the number line, and that includes all positive numbers, zero, and negative numbers. However, the square root of a negative number is not on this number line; instead, it is what we call an imaginary number. When we want to take the square root of a negative number, we typically use the imaginary unit \(i\), where \(i\) is defined as \(\sqrt{-1}\).

For the second question, the correct answer is "108 degrees." In a regular pentagon, which is a five-sided polygon with equal sides and angles, each interior angle is \(108\) degrees. This is because the sum of the interior angles of a pentagon is \(540\) degrees, and when divided by \(5\) (the number of angles),

Woohoo 🎉


## Conclusion

We covered a lot of ground in this notebook, give yourself a high-five! Hopefully you should now have a strong foundation to build powerful, stateful experiences with tools like Code Interpreter, Retrieval, and Functions!

There's a few sections we didn't cover for the sake of brevity, so here's a few resources to explore further:

- [Annotations](https://platform.openai.com/docs/assistants/how-it-works/managing-threads-and-messages): parsing file citations
- [Files](https://platform.openai.com/docs/api-reference/assistants/file-object): Thread scoped vs Assistant scoped
- [Parallel Function Calls](https://platform.openai.com/docs/guides/function-calling/parallel-function-calling): calling multiple tools in a single Step
- Multi-Assistant Thread Runs: single Thread with Messages from multiple Assistants
- Streaming: coming soon!

Now go off and build something ama[zing](https://www.youtube.com/watch?v=xvFZjo5PgG0&pp=ygUQcmljayByb2xsIG5vIGFkcw%3D%3D)!
